In [13]:
#Modelo 3
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# 1. Descargar datos históricos del activo
activo = "AAPL"  # Ticker del activo
periodo = "1y"
datos = yf.Ticker(activo).history(period=periodo)
Datos_Mercado = yf.Ticker("SPY").history(period=periodo)  

# 2. Calcular los retornos promedio (Average) y la desviación estándar (std)
datos['Average'] = datos[['High', 'Low']].mean(axis=1).pct_change()
datos['std'] = datos['Average'].rolling(window=5).std()
Datos_Mercado['RMercado'] = Datos_Mercado['Close'].pct_change()
datos['log_std'] = np.log(datos['std'])
datos['log_Average'] = np.log(datos['Average'].replace(0, np.nan).dropna())

# Eliminar valores NaN para los ajustes
datos.dropna(inplace=True)
Datos_Mercado.dropna(inplace=True)

# Asegurar que los datos tengan la misma longitud
min_length = min(len(datos['Average']), len(Datos_Mercado['RMercado']))
datos_trimmed = datos['Average'][:min_length]
mercado_trimmed = Datos_Mercado['RMercado'][:min_length]

# Coeficientes del modelos
alpha_poly_log = modelo_poly_log.params[0] # Intercepto
cov_matrix = np.cov(datos_trimmed, mercado_trimmed)
cov_am = cov_matrix[0, 1]
var_m = cov_matrix[1, 1]
beta_1_poly_log = cov_am / var_m * modelo_poly_log.params[1]
beta_2_poly_log = cov_am / var_m * modelo_poly_log.params[2]


# --- MODELO EXPONENCIAL CON LOGARITMOS ---
# Ajustar el modelo exponencial transformado con logaritmos
X_exp_log = sm.add_constant(datos['Average'])  # Log(σ) = Log(α) + β * Average
modelo_exp_log = sm.OLS(datos['log_std'], X_exp_log).fit()

# Coeficientes del modelo exponencial con logaritmos
alpha_exp_log = np.exp(modelo_exp_log.params[0])  # e^(Intercepto)
beta_exp_log = modelo_exp_log.params[1]

# Predicciones del modelo exponencial con logaritmos
datos['log_std_pred_exp'] = modelo_exp_log.params[0] + beta_exp_log * datos['Average']

# --- GRAFICAR LOS RESULTADOS ---
fig = go.Figure()

# Datos observados
fig.add_trace(go.Scatter(x=datos['Average'], y=datos['std'],
                         mode='markers', name='Datos Observados',
                         marker=dict(size=8, color='blue', opacity=0.7)))


# Línea del modelo exponencial
y_vals_exp = np.exp(modelo_exp_log.params[0] + beta_exp_log * x_vals)
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_exp, mode='lines', name='Modelo Exponencial',
                         line=dict(color='green')))

# Configuración del gráfico
fig.update_layout(title="Modelos Polinomial y Exponencial | Desv. Estándar vs Retornos Promedio",
                  xaxis_title="Retornos Promedio (Average)",
                  yaxis_title="Desviación Estándar (std)",
                  template="plotly_white")

fig.show()

# --- RESULTADOS ---
print("Resultados del Modelo Polinomial:")
print(f"Intercepto (α): {y_vals_poly[0]:.4f}")
print(f"Intercepto (α): {alpha_poly_log:.4f}")
print(f"Coeficiente Lineal (β1): {beta_1_poly_log:.4f}")
print(f"Coeficiente Cuadrático (β2): {beta_2_poly_log:.4f}")
print(f"Resumen del Modelo Polinomial:\n{modelo_poly_log.summary()}")

print("\nResultados del Modelo Exponencial:")
print(f"Intercepto (α): {alpha_exp_log:.4f}")
print(f"Coeficiente Exponencial (β): {beta_exp_log:.4f}")


C:\Users\retir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log

C:\Users\retir\AppData\Local\Temp\ipykernel_5488\2973994910.py:32: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\retir\AppData\Local\Temp\ipykernel_5488\2973994910.py:36: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\retir\AppData\Local\Temp\ipykernel_5488\2973994910.py:37: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys 

Resultados del Modelo Polinomial:
Intercepto (α): -2.0865
Intercepto (α): -2.0865
Coeficiente Lineal (β1): -0.0651
Coeficiente Cuadrático (β2): -0.0048
Resumen del Modelo Polinomial:
                            OLS Regression Results                            
Dep. Variable:                log_std   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     7.791
Date:                Sun, 26 Jan 2025   Prob (F-statistic):           0.000628
Time:                        22:50:37   Log-Likelihood:                -114.91
No. Observations:                 138   AIC:                             235.8
Df Residuals:                     135   BIC:                             244.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    st